# 前向传播、反向传播和计算图
:label:`sec_backprop`

到目前为止，我们已经使用小批量随机梯度下降训练了我们的模型。然而，在实现算法时，我们只关心通过模型的*前向传播*涉及的计算。当需要计算梯度时，我们只是调用了深度学习框架提供的反向传播函数。

自动计算梯度极大地简化了深度学习算法的实现。在自动微分之前，即使对复杂模型进行微小更改也需要手动重新计算复杂的导数。令人惊讶的是，学术论文经常需要分配大量篇幅来推导更新规则。虽然我们必须继续依赖自动微分以便专注于有趣的部分，但如果你想超越对深度学习的浅层理解，你应该知道这些梯度是如何在幕后计算的。

在本节中，我们将深入探讨*反向传播*（更常见的名称是*反向传播*）的细节。为了传达一些技巧及其实现的见解，我们依靠一些基本的数学和计算图。首先，我们将重点放在一个包含权重衰减（$\ell_2$正则化，将在后续章节中描述）的单隐藏层多层感知机上。

## 前向传播

*前向传播*（或*前向传递*）是指从输入层到输出层顺序计算并存储神经网络中的中间变量（包括输出）。我们现在逐步解析一个具有单个隐藏层的神经网络的工作机制。这可能看起来很繁琐，但正如放克大师詹姆斯·布朗的永恒名言：“你必须付出代价才能成为老板”。

为了简单起见，假设输入示例为$\mathbf{x}\in \mathbb{R}^d$，并且我们的隐藏层不包括偏置项。这里的中间变量是：

$$\mathbf{z}= \mathbf{W}^{(1)} \mathbf{x},$$

其中$\mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$是隐藏层的权重参数。将中间变量$\mathbf{z}\in \mathbb{R}^h$通过激活函数$\phi$后，我们得到长度为$h$的隐藏激活向量：

$$\mathbf{h}= \phi (\mathbf{z}).$$

隐藏层输出$\mathbf{h}$也是一个中间变量。假设输出层的参数只有权重$\mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$，我们可以得到一个长度为$q$的输出层变量：

$$\mathbf{o}= \mathbf{W}^{(2)} \mathbf{h}.$$

假设损失函数为$l$，示例标签为$y$，我们可以计算单个数据示例的损失项，

$$L = l(\mathbf{o}, y).$$

正如我们将在稍后介绍的$\ell_2$正则化的定义中看到的那样，给定超参数$\lambda$，正则化项是

$$s = \frac{\lambda}{2} \left(\|\mathbf{W}^{(1)}\|_\textrm{F}^2 + \|\mathbf{W}^{(2)}\|_\textrm{F}^2\right),$$
:eqlabel:`eq_forward-s`

其中矩阵的Frobenius范数仅仅是将矩阵展平成向量后的$\ell_2$范数。最后，模型在给定数据示例上的正则化损失为：

$$J = L + s.$$

在接下来的讨论中，我们将$J$称为*目标函数*。


## 前向传播的计算图

绘制*计算图*有助于我们可视化计算中操作符和变量之间的依赖关系。:numref:`fig_forward`包含了上面描述的简单网络的图形，其中方块表示变量，圆圈表示操作符。左下角表示输入，右上角是输出。请注意，箭头的方向（说明数据流）主要是向右和向上。

![前向传播的计算图。](../img/forward.svg)
:label:`fig_forward`

## 反向传播

*反向传播*指的是计算神经网络参数梯度的方法。简而言之，该方法根据微积分中的*链式法则*，从输出层到输入层逆序遍历网络。算法在计算相对于某些参数的梯度时，会存储所需的任何中间变量（偏导数）。假设我们有函数$\mathsf{Y}=f(\mathsf{X})$和$\mathsf{Z}=g(\mathsf{Y})$，其中输入和输出$\mathsf{X}, \mathsf{Y}, \mathsf{Z}$是任意形状的张量。通过使用链式法则，我们可以通过

$$\frac{\partial \mathsf{Z}}{\partial \mathsf{X}} = \textrm{prod}\left(\frac{\partial \mathsf{Z}}{\partial \mathsf{Y}}, \frac{\partial \mathsf{Y}}{\partial \mathsf{X}}\right).$$

这里我们使用$\textrm{prod}$操作符来乘以其参数，在必要操作（如转置和交换输入位置）完成后。对于向量来说，这是直接的：它只是矩阵-矩阵乘法。对于更高维的张量，我们使用相应的对应物。$\textrm{prod}$操作符隐藏了所有的符号开销。

回想一下，具有一个隐藏层的简单网络的参数，其计算图在:numref:`fig_forward`中，是$\mathbf{W}^{(1)}$和$\mathbf{W}^{(2)}$。反向传播的目标是计算梯度$\partial J/\partial \mathbf{W}^{(1)}$和$\partial J/\partial \mathbf{W}^{(2)}$。为此，我们应用链式法则，并依次计算每个中间变量和参数的梯度。与前向传播相比，计算顺序是相反的，因为我们需要从计算图的结果开始，朝着参数方向工作。第一步是计算目标函数$J=L+s$相对于损失项$L$和正则化项$s$的梯度：

$$\frac{\partial J}{\partial L} = 1 \; \textrm{和} \; \frac{\partial J}{\partial s} = 1.$$

接下来，我们根据链式法则计算目标函数相对于输出层变量$\mathbf{o}$的梯度：

$$
\frac{\partial J}{\partial \mathbf{o}}
= \textrm{prod}\left(\frac{\partial J}{\partial L}, \frac{\partial L}{\partial \mathbf{o}}\right)
= \frac{\partial L}{\partial \mathbf{o}}
\in \mathbb{R}^q.
$$

接下来，我们计算正则化项相对于两个参数的梯度：

$$\frac{\partial s}{\partial \mathbf{W}^{(1)}} = \lambda \mathbf{W}^{(1)}
\; \textrm{和} \;
\frac{\partial s}{\partial \mathbf{W}^{(2)}} = \lambda \mathbf{W}^{(2)}.$$

现在我们能够计算最接近输出层的模型参数$\partial J/\partial \mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$的梯度。使用链式法则得到：

$$\frac{\partial J}{\partial \mathbf{W}^{(2)}}= \textrm{prod}\left(\frac{\partial J}{\partial \mathbf{o}}, \frac{\partial \mathbf{o}}{\partial \mathbf{W}^{(2)}}\right) + \textrm{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \mathbf{W}^{(2)}}\right)= \frac{\partial J}{\partial \mathbf{o}} \mathbf{h}^\top + \lambda \mathbf{W}^{(2)}.$$
:eqlabel:`eq_backprop-J-h`

要获得关于$\mathbf{W}^{(1)}$的梯度，我们需要继续沿输出层到隐藏层进行反向传播。关于隐藏层输出$\partial J/\partial \mathbf{h} \in \mathbb{R}^h$的梯度由

$$
\frac{\partial J}{\partial \mathbf{h}}
= \textrm{prod}\left(\frac{\partial J}{\partial \mathbf{o}}, \frac{\partial \mathbf{o}}{\partial \mathbf{h}}\right)
= {\mathbf{W}^{(2)}}^\top \frac{\partial J}{\partial \mathbf{o}}.
$$

由于激活函数$\phi$逐元素应用，计算中间变量$\mathbf{z}$的梯度$\partial J/\partial \mathbf{z} \in \mathbb{R}^h$需要使用逐元素乘法操作符，我们用$\odot$表示：

$$
\frac{\partial J}{\partial \mathbf{z}}
= \textrm{prod}\left(\frac{\partial J}{\partial \mathbf{h}}, \frac{\partial \mathbf{h}}{\partial \mathbf{z}}\right)
= \frac{\partial J}{\partial \mathbf{h}} \odot \phi'\left(\mathbf{z}\right).
$$

最后，我们可以获得最接近输入层的模型参数$\partial J/\partial \mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$的梯度。根据链式法则，我们得到

$$
\frac{\partial J}{\partial \mathbf{W}^{(1)}}
= \textrm{prod}\left(\frac{\partial J}{\partial \mathbf{z}}, \frac{\partial \mathbf{z}}{\partial \mathbf{W}^{(1)}}\right) + \textrm{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \mathbf{W}^{(1)}}\right)
= \frac{\partial J}{\partial \mathbf{z}} \mathbf{x}^\top + \lambda \mathbf{W}^{(1)}.
$$



## 训练神经网络

当训练神经网络时，前向传播和反向传播相互依赖。特别是对于前向传播，我们按照依赖方向遍历计算图，并计算其路径上的所有变量。然后这些变量用于反向传播，其中计算图的顺序是相反的。

以前面提到的简单网络为例。一方面，在前向传播过程中计算正则化项:eqref:`eq_forward-s`取决于当前模型参数$\mathbf{W}^{(1)}$和$\mathbf{W}^{(2)}$的值。它们是由优化算法根据最近一次迭代的反向传播给出的。另一方面，在反向传播过程中计算参数:eqref:`eq_backprop-J-h`的梯度取决于当前隐藏层输出$\mathbf{h}$的值，这是由前向传播给出的。

因此，在训练神经网络时，一旦初始化模型参数，我们就交替进行前向传播和反向传播，使用反向传播给出的梯度更新模型参数。注意，反向传播重用了前向传播中存储的中间值，以避免重复计算。其中一个后果是我们需要保留中间值直到反向传播完成。这也是为什么训练比单纯预测需要更多内存的原因之一。此外，这种中间值的大小大致与网络层数和批量大小成正比。因此，使用更大的批量大小训练更深的网络更容易导致*内存不足*错误。


## 总结

前向传播按顺序计算并存储由神经网络定义的计算图中的中间变量。它从前向后进行。
反向传播按顺序计算并存储神经网络中中间变量和参数的梯度，顺序相反。
在训练深度学习模型时，前向传播和反向传播是相互依赖的，训练比预测需要显著更多的内存。


## 练习

1. 假设某个标量函数$f$的输入$\mathbf{X}$是$n \times m$矩阵。$f$关于$\mathbf{X}$的梯度的维度是多少？
1. 在本节描述的模型的隐藏层中添加偏置（不需要在正则化项中包含偏置）。
    1. 绘制相应的计算图。
    1. 推导前向传播和反向传播的方程。
1. 计算本节描述的模型在训练和预测中的内存占用。
1. 假设你想计算二阶导数。计算图会发生什么？你预计计算需要多长时间？
1. 假设计算图太大，无法放入你的GPU。
    1. 你能将其分割到多个GPU上吗？
    1. 相对于在较小的小批量上训练，这样做有什么优缺点？

[讨论](https://discuss.d2l.ai/t/102)